In [72]:
import pandas as pd
import numpy as np
import json
import requests

In [73]:
headers = {"User-Agent" : "ceellam45@gmail.com"}

In [74]:
company_tickers = requests.get("https://www.sec.gov/files/company_tickers.json",
             headers=headers
            )

In [75]:
company_data = pd.DataFrame.from_dict(company_tickers.json(),orient='index')

In [76]:
company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)
company_data

,cik_str,ticker,title
0,0000789019,MSFT,MICROSOFT CORP
1,0000320193,AAPL,Apple Inc.
2,0001045810,NVDA,NVIDIA CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC
...,...,...,...
10389,0001861121,EVE-WT,EVe Mobility Acquisition Corp
10390,0001969475,BAYAU,Bayview Acquisition Corp
10391,0001969475,BAYAR,Bayview Acquisition Corp
10392,0000748954,ATLFF,ATLAS COPCO AB


In [77]:
cik = company_data.iloc[89]['cik_str']

cik

'0000310764'

In [78]:
filing_data = requests.get(f"https://data.sec.gov/submissions/CIK{cik}.json",
             headers=headers
            )
filing_data.json()['filings']['recent'].keys()

dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])

In [79]:
allforms = pd.DataFrame.from_dict(filing_data.json()['filings']['recent'])

allforms = allforms[['accessionNumber','reportDate','form']]
allforms = allforms[allforms['form'] == '10-K']
allforms

,accessionNumber,reportDate,form
21,0000310764-24-000024,2023-12-31,10-K
150,0000310764-23-000017,2022-12-31,10-K
222,0000310764-22-000028,2021-12-31,10-K
307,0000310764-21-000027,2020-12-31,10-K
427,0000310764-20-000014,2019-12-31,10-K
539,0000310764-19-000014,2018-12-31,10-K
664,0000310764-18-000031,2017-12-31,10-K
756,0000310764-17-000034,2016-12-31,10-K
970,0000310764-16-000226,2015-12-31,10-K


In [80]:
companyfacts = requests.get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json",
             headers=headers
            )
companyfacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'][-1:]


[{'end': '2024-01-31',
  'val': 380264036,
  'accn': '0000310764-24-000024',
  'fy': 2023,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2024-02-14',
  'frame': 'CY2023Q4I'}]

In [81]:
#companyfacts.json()['facts']['us-gaap']['Revenues']['units']['USD']



In [83]:
data = companyfacts.json()['facts']['us-gaap']['Revenues']['units']['USD']

df = pd.DataFrame.from_dict(data)

df

,start,end,val,accn,fy,fp,form,filed,frame
0,2016-01-01,2016-12-31,11325000000,0000310764-19-000014,2018,FY,10-K,2019-02-07,CY2016
1,2017-01-01,2017-03-31,2955000000,0000310764-19-000014,2018,FY,10-K,2019-02-07,CY2017Q1
2,2017-04-01,2017-06-30,3012000000,0000310764-19-000014,2018,FY,10-K,2019-02-07,CY2017Q2
3,2017-01-01,2017-09-30,8973000000,0000310764-18-000205,2018,Q3,10-Q,2018-10-26,NaN
4,2017-07-01,2017-09-30,3006000000,0000310764-18-000205,2018,Q3,10-Q,2018-10-26,NaN
...,...,...,...,...,...,...,...,...,...
115,2023-01-01,2023-09-30,14683000000,0000310764-23-000116,2023,Q3,10-Q,2023-11-03,NaN
116,2023-07-01,2023-09-30,4909000000,0000310764-23-000116,2023,Q3,10-Q,2023-11-03,NaN
117,2023-07-01,2023-09-30,4909000000,0000310764-24-000024,2023,FY,10-K,2024-02-14,CY2023Q3
118,2023-01-01,2023-12-31,20498000000,0000310764-24-000024,2023,FY,10-K,2024-02-14,CY2023
